## BESTPOS ASCII TO CSV CONVERSION

In [9]:
import os
import datetime
import pytz
import csv

# Number of leap seconds to subtract from GPS time (18 seconds as of current)
LEAP_SECONDS = 18

# GPS epoch start date
GPS_EPOCH = datetime.datetime(1980, 1, 6, tzinfo=datetime.timezone.utc)

# IST timezone
IST = pytz.timezone('Asia/Kolkata')

def gps_to_ist(gps_week, tow):
    """
    Convert GPS week and time of week to IST.
    """
    # Total seconds since GPS epoch
    total_seconds = gps_week * 7 * 24 * 3600 + tow - LEAP_SECONDS
    # Convert to UTC
    utc_time = GPS_EPOCH + datetime.timedelta(seconds=total_seconds)
    # Convert to IST
    ist_time = utc_time.astimezone(IST)
    return ist_time

# File paths
input_file_path = "/Users/ravitejakunchanapalli/Desktop/testing_data/BESTPOS.ASCII"
output_file_path = "/Users/ravitejakunchanapalli/Downloads/csvdata/bestposdata.csv"

# Ensure the output directory exists
output_directory = os.path.dirname(output_file_path)
os.makedirs(output_directory, exist_ok=True)

# Initialize lists to store parsed data
latitudes = []
longitudes = []
ist_times = []

# Read the ASCII file and extract relevant fields
with open(input_file_path, 'r') as file:
    data = file.readlines()

# Iterate through each line in the file
for line in data:
    # Split the line into log header and fields
    log_header, fields = line.split(';')
    log_header = log_header.split(',')
    week = int(log_header[5])  # Assuming GPS week is at index 5
    tow = float(log_header[6])  # Assuming time of week is at index 6
    
    # Split the fields into individual data points
    fields = fields.split(',')

    # Extracting relevant data (latitude, longitude, and height)
    lat = float(fields[2])
    lon = float(fields[3])

    # Convert GPS week and time of week to IST time
    ist_time = gps_to_ist(week, tow).strftime('%Y/%m/%d/%H:%M:%S.%f')  # Format to string

    # Store the parsed data in lists
    latitudes.append(lat)
    longitudes.append(lon)
    ist_times.append(ist_time)

# Write the parsed data to a new CSV file
with open(output_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write header
    csvwriter.writerow(['time', 'latitude', 'longitude'])
    # Write data
    for ist_time, lat, lon in zip(ist_times, latitudes, longitudes):
        csvwriter.writerow([ist_time, lat, lon])

print(f"Data has been parsed and saved to {output_file_path}")


Data has been parsed and saved to /Users/ravitejakunchanapalli/Downloads/csvdata/bestposdata.csv


## BESTVEL ASCII TO CSV CONVERSION

In [10]:
import os
import datetime
import pytz
import pandas as pd

# Number of leap seconds to subtract from GPS time (18 seconds as of current)
LEAP_SECONDS = 18

# GPS epoch start date
GPS_EPOCH = datetime.datetime(1980, 1, 6, tzinfo=datetime.timezone.utc)

# IST timezone
IST = pytz.timezone('Asia/Kolkata')

def gps_to_ist(gps_week, tow):
    """
    Convert GPS week and time of week to IST.
    """
    # Total seconds since GPS epoch
    total_seconds = gps_week * 7 * 24 * 3600 + tow - LEAP_SECONDS
    # Convert to UTC
    utc_time = GPS_EPOCH + datetime.timedelta(seconds=total_seconds)
    # Convert to IST
    ist_time = utc_time.astimezone(IST)
    return ist_time

def parse_bestvel_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('#BESTVELA'):
                log_header, fields = line.split(';')
                log_header = log_header.split(',')
                week = int(log_header[5])  # Assuming GPS week is at index 5
                tow = float(log_header[6])  # Assuming time of week is at index 6
                
                fields = fields.split(',')
                hor_speed = float(fields[4]) * 3.6  # Assuming hor_speed is at index 4 and convert to km/h
               

                # Convert GPS week and time of week to IST time
                ist_time = gps_to_ist(week, tow).strftime('%Y/%m/%d/%H:%M:%S.%f')  # Format to string
                
                # Append the data to the list
                data.append([ist_time, hor_speed])

    return pd.DataFrame(data, columns=['time', 'horizontal_speed'])

# File paths
input_file_path = "/Users/ravitejakunchanapalli/Desktop/testing_data/BESTVEL.ASCII"
output_file_path = "/Users/ravitejakunchanapalli/Downloads/csvdata/bestveldata.csv"

# Ensure the output directory exists
output_directory = os.path.dirname(output_file_path)
os.makedirs(output_directory, exist_ok=True)

# Parse the bestvel data
df = parse_bestvel_data(input_file_path)

# Save the data to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Data has been parsed and saved to {output_file_path}")


Data has been parsed and saved to /Users/ravitejakunchanapalli/Downloads/csvdata/bestveldata.csv


## Images Fitting code

In [ ]:
import pandas as pd
import folium
import os
from matplotlib import cm, colors
import matplotlib.pyplot as plt

# Load the GNSS data
gnss_file_path = '/Users/ravitejakunchanapalli/Downloads/baleno_data/gnss_data.csv'
gnss_df = pd.read_csv(gnss_file_path)
gnss_df['time'] = pd.to_datetime(gnss_df['time'])

# Load the BestVel data
bestvel_file_path = '/Users/ravitejakunchanapalli/Downloads/baleno_data/bestvel_data.csv'
bestvel_df = pd.read_csv(bestvel_file_path)
bestvel_df['time'] = pd.to_datetime(bestvel_df['time'])

# Load the Basler data
basler_file_path = '/Users/ravitejakunchanapalli/Downloads/baleno_data/basler_data.csv'
basler_df = pd.read_csv(basler_file_path)
basler_df['time'] = pd.to_datetime(basler_df['time'])

# Merge Basler data with GNSS data based on nearest timestamp from GNSS data
merged_data = pd.merge_asof(gnss_df, basler_df, on='time')

# Merge with BestVel data based on nearest timestamp from BestVel data
merged_data = pd.merge_asof(bestvel_df, merged_data, on='time')

# Directory containing images
image_dir = '/Users/ravitejakunchanapalli/Downloads/baleno_images'

# Function to get image filename based on time
def get_nearest_image(time):
    image_timestamps = pd.to_datetime(os.listdir(image_dir), format='image_%Y-%m-%d-%H-%M-%S.%f.jpg')
    nearest_image = min(image_timestamps, key=lambda x: abs(x - time))
    return f'image_{nearest_image.strftime("%Y-%m-%d-%H-%M-%S.%f")}.jpg'

# Attach nearest image filename to merged data
merged_data['image_filename'] = merged_data['time'].apply(get_nearest_image)

print("len",len(merged_data))
print(merged_data.head())
# Normalize speed data for color mapping
norm = colors.Normalize(vmin=merged_data['horizontal_speed'].min(), vmax=merged_data['horizontal_speed'].max())
cmap = plt.get_cmap('RdYlGn_r')

# Create the map
m = folium.Map(
    location=[merged_data['latitude'].mean(), merged_data['longitude'].mean()],
    zoom_start=18,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google'
)

# Add points to the map with color-coded markers and popups based on speed
for _, row in merged_data.iterrows():
    color = colors.rgb2hex(cmap(norm(row['horizontal_speed'])))
    popup_text = f"Latitude: {row['latitude']:.6f}<br>Longitude: {row['longitude']:.6f}<br>Speed: {row['horizontal_speed']:.2f}"
    image_path = os.path.join(image_dir, row['image_filename'])
    if os.path.exists(image_path):
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=folium.Popup(popup_text + f"<br><img src='{image_path}' width='200'>", max_width=300)
        ).add_to(m)
    else:
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=folium.Popup(popup_text, max_width=200)
        ).add_to(m)

# Save the map as an HTML file
map_path = '/Users/ravitejakunchanapalli/Desktop/img_test6.html'
m.save(map_path)
print(f'Map saved to {map_path}')
# Create a color bar and save as an image
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=ax, orientation='horizontal')
cbar.set_label('Speed (km/h)')

colorbar_path = '/Users/ravitejakunchanapalli/Desktop/colorbar6.png'
plt.savefig(colorbar_path)
print(f'Color bar saved to {colorbar_path}')

